<a href="https://colab.research.google.com/github/chandan110791/NLP/blob/main/spell_corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re
from collections import Counter

In [5]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [6]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('big.txt').read()))

In [7]:
# check frequency of a random word, say, 'chair'
all_words['chair']

64

In [8]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 28193),
 ('of', 15352),
 ('and', 10451),
 ('to', 8784),
 ('in', 8209),
 ('a', 7079),
 ('was', 3986),
 ('that', 3474),
 ('it', 3468),
 ('i', 3145)]

In [9]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [10]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [11]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

In [12]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [13]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [14]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'fonney', 'monnxy', 'mononey', 'rmonney', 'mmonney', 'monneky', 'mognney', 'monneyn', 'mbonney', 'ponney', 'mzonney', 'manney', 'gonney', 'mponney', 'omonney', 'montey', 'mronney', 'monndy', 'conney', 'mondey', 'monzey', 'bmonney', 'moncney', 'wonney', 'mmnney', 'lonney', 'wmonney', 'eonney', 'monneny', 'honney', 'molnney', 'monqney', 'monneyr', 'monnyey', 'mhnney', 'monjey', 'moeney', 'mobnney', 'momnney', 'moyney', 'amonney', 'monnyy', 'monnpy', 'ymonney', 'monnea', 'monniey', 'monwey', 'monnley', 'monneyt', 'vonney', 'movney', 'monnvy', 'hmonney', 'monnehy', 'monneyd', 'monny', 'monnefy', 'monnhy', 'monnep', 'mosnney', 'monmney', 'monneyq', 'moncey', 'moneny', 'monhney', 'monxney', 'monnjy', 'moanney', 'monnezy', 'monnoey', 'monaey', 'emonney', 'yonney', 'monnby', 'monnee', 'sonney', 'monneh', 'monneyb', 'money', 'mvnney', 'monnmy', 'monvney', 'monnemy', 'mogney', 'mobney', 'mqnney', 'monnny', 'mojnney', 'monhey', 'konney', 'mknney', 'dmonney', 'meonney', 'mocnney', 'monvey', '

In [25]:
len(set(edits_two("emfasize")))

90902

In [15]:
print(known(edits_one("monney")))

{'money'}


In [16]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money'}


In [17]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money'}


In [26]:
print(possible_corrections("emfasize"))

{'emphasize'}


In [18]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0005374100266989875
0.0


In [19]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [20]:
# test spell check
print(spell_check("monney"))

Did you mean money?
